В этой программе я достаю из корпуса все глаголы и предложные группы, которые встречаются после *давай*, *пойдем* и *го*. 

In [1]:
import sqlite3
from collections import Counter
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [2]:
conn = sqlite3.connect('oleg.db')
cur = conn.cursor()

In [3]:
def nothing():
    return

In [4]:
cur.execute('''
select text
from texts
join meta on texts.id = meta.id
where instr(text, 'давай') != 0 or instr(text, 'Давай') != 0 
or instr(text, 'го') != 0 or instr(text, 'Го') != 0 
or instr(text, 'пойдем') != 0 or instr(text, 'Пойдем') != 0 
or instr(text, 'пойдём') != 0 or instr(text, 'Пойдём') != 0''')

mess = cur.fetchall()

In [5]:
def is_right(s):
    s = s.lower().split()
    
    if 'давай' in s:
        i = s.index('давай')
    elif 'го' in s:
        i = s.index('го')
    elif 'пойдем' in s:
        i = s.index('пойдем')
    elif 'пойдём' in s:
        i = s.index('пойдём')
    else:
        return
    
    try:
        pos = morph.parse(s[i+1])[0].tag.POS
        lemma = morph.parse(s[i+1])[0].normal_form
    except IndexError:
        pos = 0
    
    if pos == 'INFN' or pos == 'VERB':
        return lemma
    elif pos == 'PREP':
        lemma1 = lemma
        pos2 = morph.parse(s[i+2])[0].tag.POS
        lemma2 = morph.parse(s[i+2])[0].normal_form
        if pos2 == 'NOUN':
            a = lemma1 + ' ' + s[i+2]
            return a

In [6]:
def get_words(mess):
    words = []
    for el in mess:
        word = is_right(el[0])
        if word:
            words.append(word)
    return words

In [7]:
def get_cnt(words):
    return Counter(words).most_common()

In [8]:
def m(mess):
    words = get_words(mess)
    cnt = get_cnt(words)
    return words, cnt

Результатов оказалось неожиданно мало:

In [9]:
m(mess)[1]

[('гулять', 6),
 ('посмотреть', 5),
 ('кататься', 2),
 ('остановиться', 2),
 ('поговорить', 2),
 ('решать', 2),
 ('сделать', 2),
 ('попробовать', 2),
 ('в квартиру', 1),
 ('помочь', 1),
 ('мочь', 1),
 ('в воскресенье', 1),
 ('в пн', 1),
 ('играть', 1),
 ('в париж', 1),
 ('прикинуть', 1),
 ('на маяковской', 1),
 ('кушать', 1),
 ('выучить', 1),
 ('через час', 1),
 ('в вс', 1),
 ('в сторону', 1),
 ('в половину', 1),
 ('до полуночи', 1),
 ('понять', 1),
 ('на улицу', 1),
 ('лазать', 1),
 ('подумать', 1),
 ('в тулузу', 1),
 ('в парк', 1),
 ('встать', 1),
 ('в зоопарк', 1),
 ('на пламя', 1),
 ('в москве', 1),
 ('сходить', 1),
 ('пойти', 1),
 ('в пятницу', 1),
 ('смотреть', 1),
 ('дособирать', 1),
 ('встретиться', 1),
 ('про безответственность', 1),
 ('договориться', 1),
 ('есть', 1),
 ('выбрать', 1),
 ('селиться', 1),
 ('в воскреснье', 1),
 ('покупать', 1),
 ('к канту', 1),
 ('в час', 1)]